In [ ]:
import math

import torch
from pytorch_pretrained_bert import BertTokenizer, BertForMaskedLM
from tqdm import tqdm_notebook

In [ ]:
bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
def get_score(sentence: str):
    tokenize_input = bert_tokenizer.tokenize(sentence)
    tensor_input = torch.tensor([bert_tokenizer.convert_tokens_to_ids(tokenize_input)])
    predictions = bert_model(tensor_input)
    loss_fct = torch.nn.CrossEntropyLoss()
    loss = loss_fct(predictions.squeeze(), tensor_input.squeeze()).data
    return math.exp(loss)

def sorted_tuple_scores(tuple_data_path: str):
    scores = []
    with open(tuple_data_path, 'r') as f:
        for line in tqdm_notebook(f.readlines()):
            sentence = " ".join(line.strip().split('\t'))
            scores.append((sentence, get_score(sentence)))
    scores.sort(key=lambda x: x[1])
    return scores

In [ ]:
aristo_scores = sorted_tuple_scores('data/aristo.txt')
display(aristo_scores)

In [ ]:
openbook_scores = sorted_tuple_scores('data/openbook_tuples.tsv')
display(openbook_scores)